# Jour 3 - Exercice 2 : Ensemble Learning pour la Classification

## Objectifs
- Comprendre et appliquer les techniques d'Ensemble Learning pour la classification
- Implémenter différentes méthodes d'ensemble : Bagging, Boosting et Stacking
- Évaluer les performances des modèles sur le jeu de test
- Visualiser et interpréter les résultats avec Plotly
- Comparer les performances des différentes approches d'ensemble

## Introduction

Dans ce notebook, nous allons explorer les techniques d'Ensemble Learning pour améliorer les performances de nos modèles de classification. L'Ensemble Learning consiste à combiner plusieurs modèles pour obtenir de meilleures prédictions que celles obtenues avec un seul modèle. Nous utiliserons les données de satisfaction des passagers et appliquerons différentes techniques d'ensemble pour prédire si un passager est satisfait ou non de son expérience de vol.

## 1. Importation des bibliothèques et chargement des données

In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import pickle
import sys
import os

# Visualisation avec Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Scikit-learn pour le machine learning
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, roc_curve, auc

# Modèles de base
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Modèles d'ensemble
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.ensemble import StackingClassifier

# Ajouter le chemin pour importer utils.py
sys.path.append(os.path.abspath('./'))
from utils import results_predictions_satisfaction

# Pour afficher plus de colonnes dans les DataFrames
pd.set_option('display.max_columns', 30)

In [ ]:
# Chargement des données d'entraînement
train_df = pd.read_csv('../../data/passenger_satisfaction/train.csv')

# Affichage des premières lignes
train_df.head()

In [ ]:
# Chargement des données de test
test_df = pd.read_csv('../../data/passenger_satisfaction/test.csv')

# Affichage des premières lignes
test_df.head()

In [ ]:
# Vérification de la distribution de la variable cible dans le jeu d'entraînement
target_counts = train_df['Satisfaction'].value_counts()

fig = px.pie(values=target_counts.values, 
             names=target_counts.index, 
             title='Distribution de la satisfaction des passagers (Train)',
             color_discrete_sequence=px.colors.qualitative.Set3)

fig.update_traces(textinfo='percent+label')
fig.show()

## 2. Préparation des données

Nous allons préparer nos données pour l'entraînement des modèles d'ensemble. Cela comprend le nettoyage des données, la gestion des valeurs manquantes, l'encodage des variables catégorielles et la normalisation des variables numériques.

In [ ]:
# Vérification des valeurs manquantes dans le jeu d'entraînement
missing_values = train_df.isnull().sum()
missing_values = missing_values[missing_values > 0]

if len(missing_values) > 0:
    print("Valeurs manquantes dans le jeu d'entraînement :")
    print(missing_values)
else:
    print("Aucune valeur manquante dans le jeu d'entraînement.")

In [ ]:
# Vérification des valeurs manquantes dans le jeu de test
missing_values_test = test_df.isnull().sum()
missing_values_test = missing_values_test[missing_values_test > 0]

if len(missing_values_test) > 0:
    print("Valeurs manquantes dans le jeu de test :")
    print(missing_values_test)
else:
    print("Aucune valeur manquante dans le jeu de test.")

In [7]:
# Suppression de la colonne ID qui n'est pas utile pour la prédiction
train_df = train_df.drop('ID', axis=1)
test_df = test_df.drop('ID', axis=1)

In [8]:
# Séparation des features et de la variable cible pour le jeu d'entraînement
X_train = train_df.drop('Satisfaction', axis=1)
y_train = train_df['Satisfaction']

# Préparation du jeu de test
X_test = test_df.copy()

In [ ]:
# Identification des colonnes numériques et catégorielles
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

print(f"Features numériques : {numeric_features}")
print(f"Features catégorielles : {categorical_features}")

In [10]:
# Création d'un pipeline de prétraitement
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Application du prétraitement aux données d'entraînement
X_train_preprocessed = preprocessor.fit_transform(X_train)

# Application du même prétraitement aux données de test
X_test_preprocessed = preprocessor.transform(X_test)

In [ ]:
# Vérification de la forme des données prétraitées
print(f"Forme de X_train_preprocessed : {X_train_preprocessed.shape}")
print(f"Forme de X_test_preprocessed : {X_test_preprocessed.shape}")

In [ ]:
# Création d'un jeu de validation à partir du jeu d'entraînement
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_preprocessed, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"Forme de X_train_final : {X_train_final.shape}")
print(f"Forme de X_val : {X_val.shape}")

## 3. Implémentation des modèles d'Ensemble Learning

Dans cette section, nous allons implémenter différentes techniques d'Ensemble Learning pour la classification :
- Bagging (Random Forest, Bagging Classifier)
- Boosting (AdaBoost, Gradient Boosting)
- Voting (Hard et Soft Voting)
- Stacking

### 3.1 Bagging (Random Forest)

In [ ]:
# Implémentation du Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_final, y_train_final)

# Prédictions sur le jeu de validation
y_val_pred_rf = rf_model.predict(X_val)

# Évaluation des performances
print("Performances du Random Forest sur le jeu de validation :")
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_rf):.4f}")
print(f"F1 Score: {f1_score(y_val, y_val_pred_rf, pos_label='satisfied'):.4f}")
print("\nRapport de classification :")
print(classification_report(y_val, y_val_pred_rf))

In [ ]:
# Visualisation de la matrice de confusion pour le Random Forest
conf_matrix_rf = confusion_matrix(y_val, y_val_pred_rf)

# Création de la figure avec Plotly
fig = px.imshow(conf_matrix_rf,
                labels=dict(x="Prédiction", y="Réalité", color="Nombre"),
                x=['neutral or dissatisfied', 'satisfied'],
                y=['neutral or dissatisfied', 'satisfied'],
                title="Matrice de confusion - Random Forest",
                color_continuous_scale='Blues')

# Ajout des annotations
for i in range(len(conf_matrix_rf)):
    for j in range(len(conf_matrix_rf[i])):
        fig.add_annotation(x=j, y=i, 
                          text=str(conf_matrix_rf[i, j]),
                          showarrow=False,
                          font=dict(color="white" if conf_matrix_rf[i, j] > conf_matrix_rf.max()/2 else "black"))

fig.show()

In [ ]:
# Implémentation du Bagging Classifier avec un arbre de décision comme estimateur de base
bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(), 
                                 n_estimators=100, 
                                 random_state=42)
bagging_model.fit(X_train_final, y_train_final)

# Prédictions sur le jeu de validation
y_val_pred_bagging = bagging_model.predict(X_val)

# Évaluation des performances
print("Performances du Bagging Classifier sur le jeu de validation :")
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_bagging):.4f}")
print(f"F1 Score: {f1_score(y_val, y_val_pred_bagging, pos_label='satisfied'):.4f}")
print("\nRapport de classification :")
print(classification_report(y_val, y_val_pred_bagging))

### 3.2 Boosting (AdaBoost, Gradient Boosting)

In [ ]:
# Implémentation de l'AdaBoost
adaboost_model = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1), 
                                   n_estimators=100, 
                                   random_state=42)
adaboost_model.fit(X_train_final, y_train_final)

# Prédictions sur le jeu de validation
y_val_pred_adaboost = adaboost_model.predict(X_val)

# Évaluation des performances
print("Performances de l'AdaBoost sur le jeu de validation :")
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_adaboost):.4f}")
print(f"F1 Score: {f1_score(y_val, y_val_pred_adaboost, pos_label='satisfied'):.4f}")
print("\nRapport de classification :")
print(classification_report(y_val, y_val_pred_adaboost))

In [ ]:
# Implémentation du Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, 
                                     learning_rate=0.1, 
                                     max_depth=3, 
                                     random_state=42)
gb_model.fit(X_train_final, y_train_final)

# Prédictions sur le jeu de validation
y_val_pred_gb = gb_model.predict(X_val)

# Évaluation des performances
print("Performances du Gradient Boosting sur le jeu de validation :")
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_gb):.4f}")
print(f"F1 Score: {f1_score(y_val, y_val_pred_gb, pos_label='satisfied'):.4f}")
print("\nRapport de classification :")
print(classification_report(y_val, y_val_pred_gb))

### 3.3 Voting Classifier

In [ ]:
# Création des modèles de base pour le Voting Classifier
log_reg = LogisticRegression(max_iter=1000, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Implémentation du Hard Voting
hard_voting = VotingClassifier(
    estimators=[('lr', log_reg), ('rf', rf), ('gb', gb)],
    voting='hard'
)
hard_voting.fit(X_train_final, y_train_final)

# Prédictions sur le jeu de validation
y_val_pred_hard = hard_voting.predict(X_val)

# Évaluation des performances
print("Performances du Hard Voting sur le jeu de validation :")
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_hard):.4f}")
print(f"F1 Score: {f1_score(y_val, y_val_pred_hard, pos_label='satisfied'):.4f}")
print("\nRapport de classification :")
print(classification_report(y_val, y_val_pred_hard))

In [ ]:
# Implémentation du Soft Voting
soft_voting = VotingClassifier(
    estimators=[('lr', log_reg), ('rf', rf), ('gb', gb)],
    voting='soft'
)
soft_voting.fit(X_train_final, y_train_final)

# Prédictions sur le jeu de validation
y_val_pred_soft = soft_voting.predict(X_val)

# Évaluation des performances
print("Performances du Soft Voting sur le jeu de validation :")
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_soft):.4f}")
print(f"F1 Score: {f1_score(y_val, y_val_pred_soft, pos_label='satisfied'):.4f}")
print("\nRapport de classification :")
print(classification_report(y_val, y_val_pred_soft))

### 3.4 Stacking

In [ ]:
# Création des modèles de base pour le Stacking
estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
    ('lr', LogisticRegression(max_iter=1000, random_state=42))
]

# Implémentation du Stacking avec un modèle final de régression logistique
stacking_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1000, random_state=42),
    cv=5
)
stacking_model.fit(X_train_final, y_train_final)

# Prédictions sur le jeu de validation
y_val_pred_stacking = stacking_model.predict(X_val)

# Évaluation des performances
print("Performances du Stacking sur le jeu de validation :")
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_stacking):.4f}")
print(f"F1 Score: {f1_score(y_val, y_val_pred_stacking, pos_label='satisfied'):.4f}")
print("\nRapport de classification :")
print(classification_report(y_val, y_val_pred_stacking))

## 4. Comparaison des modèles et sélection du meilleur modèle

In [ ]:
# Création d'un DataFrame pour comparer les performances des différents modèles
models = {
    'Random Forest': (rf_model, y_val_pred_rf),
    'Bagging Classifier': (bagging_model, y_val_pred_bagging),
    'AdaBoost': (adaboost_model, y_val_pred_adaboost),
    'Gradient Boosting': (gb_model, y_val_pred_gb),
    'Hard Voting': (hard_voting, y_val_pred_hard),
    'Soft Voting': (soft_voting, y_val_pred_soft),
    'Stacking': (stacking_model, y_val_pred_stacking)
}

results = []
for model_name, (model, y_pred) in models.items():
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, pos_label='satisfied')
    results.append({
        'Modèle': model_name,
        'Accuracy': accuracy,
        'F1 Score': f1
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='F1 Score', ascending=False)
results_df

In [ ]:
# Visualisation des performances des modèles
fig = make_subplots(rows=1, cols=2, subplot_titles=('Accuracy', 'F1 Score'))

# Tri des modèles par F1 Score
sorted_results = results_df.sort_values(by='F1 Score', ascending=True)

# Ajout des barres pour l'Accuracy
fig.add_trace(
    go.Bar(
        x=sorted_results['Accuracy'],
        y=sorted_results['Modèle'],
        orientation='h',
        marker=dict(color='royalblue'),
        name='Accuracy'
    ),
    row=1, col=1
)

# Ajout des barres pour le F1 Score
fig.add_trace(
    go.Bar(
        x=sorted_results['F1 Score'],
        y=sorted_results['Modèle'],
        orientation='h',
        marker=dict(color='firebrick'),
        name='F1 Score'
    ),
    row=1, col=2
)

# Mise à jour de la mise en page
fig.update_layout(
    title_text="Comparaison des performances des modèles d'ensemble",
    height=500,
    width=1000,
    showlegend=False
)

fig.show()

## 5. Évaluation du meilleur modèle sur le jeu de test

Maintenant que nous avons comparé les performances des différents modèles sur le jeu de validation, nous allons sélectionner le meilleur modèle et l'évaluer sur le jeu de test.

In [ ]:
# Sélection du meilleur modèle (celui avec le F1 Score le plus élevé)
best_model_name = results_df.iloc[0]['Modèle']
best_model = models[best_model_name][0]

print(f"Le meilleur modèle est : {best_model_name}")

In [24]:
# Réentraînement du meilleur modèle sur l'ensemble du jeu d'entraînement
# (X_train_preprocessed et y_train)
best_model.fit(X_train_preprocessed, y_train)

# Prédictions sur le jeu de test
y_test_pred = best_model.predict(X_test_preprocessed)

In [ ]:
# Utilisation de la fonction results_predictions_satisfaction pour évaluer les performances sur le jeu de test
f1_test = results_predictions_satisfaction(y_test_pred)
print(f"F1 Score sur le jeu de test : {f1_test:.4f}")

## 6. Analyse des caractéristiques importantes

Pour certains modèles comme Random Forest et Gradient Boosting, nous pouvons extraire l'importance des caractéristiques pour comprendre quels facteurs influencent le plus la satisfaction des passagers.

In [26]:
# Vérification si le meilleur modèle permet d'extraire l'importance des caractéristiques
if hasattr(best_model, 'feature_importances_') or (hasattr(best_model, 'estimators_') and hasattr(best_model.estimators_[0], 'feature_importances_')):
    # Extraction de l'importance des caractéristiques
    if hasattr(best_model, 'feature_importances_'):
        feature_importances = best_model.feature_importances_
    else:
        # Pour les modèles d'ensemble comme VotingClassifier, on peut prendre le premier estimateur
        for estimator_name, estimator in best_model.named_estimators_.items():
            if hasattr(estimator, 'feature_importances_'):
                feature_importances = estimator.feature_importances_
                break
    
    # Récupération des noms des caractéristiques après prétraitement
    # Cela peut être complexe avec ColumnTransformer, donc nous allons utiliser une approche simplifiée
    # en utilisant les noms des colonnes originales
    
    # Création d'un DataFrame pour visualiser l'importance des caractéristiques
    if 'feature_importances_' in locals():
        # Obtention des noms de caractéristiques après one-hot encoding
        feature_names = []
        for name, transformer, features in preprocessor.transformers_:
            if name == 'cat' and hasattr(transformer.named_steps['onehot'], 'get_feature_names_out'):
                # Pour les caractéristiques catégorielles, obtenir les noms après one-hot encoding
                cat_features = transformer.named_steps['onehot'].get_feature_names_out(features)
                feature_names.extend(cat_features)
            else:
                # Pour les caractéristiques numériques, utiliser les noms d'origine
                feature_names.extend(features)
        
        # Si le nombre de noms de caractéristiques ne correspond pas au nombre d'importances
        # Utiliser des indices comme noms de caractéristiques
        if len(feature_names) != len(feature_importances):
            feature_names = [f"Feature {i}" for i in range(len(feature_importances))]
        
        # Création du DataFrame
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': feature_importances
        })
        
        # Tri par importance décroissante
        importance_df = importance_df.sort_values(by='Importance', ascending=False)
        
        # Affichage des 15 caractéristiques les plus importantes
        top_15 = importance_df.head(15)
        
        # Visualisation avec Plotly
        fig = px.bar(top_15, 
                     x='Importance', 
                     y='Feature', 
                     orientation='h',
                     title=f"Top 15 des caractéristiques les plus importantes ({best_model_name})",
                     color='Importance',
                     color_continuous_scale='Viridis')
        
        fig.update_layout(yaxis={'categoryorder':'total ascending'})
        fig.show()
else:
    print(f"Le modèle {best_model_name} ne fournit pas d'importance des caractéristiques.")

## 7. Courbes ROC et AUC

Nous allons maintenant visualiser les courbes ROC et calculer l'AUC pour les différents modèles afin de comparer leurs performances.

In [ ]:
# Création d'un encodeur pour transformer les étiquettes en valeurs numériques
le = LabelEncoder()
y_val_encoded = le.fit_transform(y_val)

# Initialisation de la figure
fig = go.Figure()

# Couleurs pour les différents modèles
colors = px.colors.qualitative.Plotly

# Calcul des courbes ROC pour chaque modèle
for i, (model_name, (model, _)) in enumerate(models.items()):
    # Vérification si le modèle peut prédire des probabilités
    if hasattr(model, 'predict_proba'):
        # Prédiction des probabilités
        y_val_proba = model.predict_proba(X_val)[:, 1]
        
        # Calcul de la courbe ROC
        fpr, tpr, _ = roc_curve(y_val_encoded, y_val_proba)
        auc_score = auc(fpr, tpr)
        
        # Ajout de la courbe ROC à la figure
        fig.add_trace(go.Scatter(
            x=fpr, y=tpr,
            name=f"{model_name} (AUC = {auc_score:.4f})",
            mode='lines',
            line=dict(color=colors[i % len(colors)], width=2)
        ))

# Ajout de la ligne de référence (random classifier)
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    name='Random Classifier',
    mode='lines',
    line=dict(color='black', width=2, dash='dash')
))

# Mise à jour de la mise en page
fig.update_layout(
    title_text="Courbes ROC pour les différents modèles d'ensemble",
    xaxis_title="Taux de faux positifs",
    yaxis_title="Taux de vrais positifs",
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255, 255, 255, 0.5)'),
    width=800,
    height=600
)

fig.show()

## 8. Conclusion

Dans ce notebook, nous avons exploré différentes techniques d'Ensemble Learning pour la classification de la satisfaction des passagers. Nous avons implémenté et comparé plusieurs approches :

1. **Bagging** : Random Forest et Bagging Classifier
2. **Boosting** : AdaBoost et Gradient Boosting
3. **Voting** : Hard Voting et Soft Voting
4. **Stacking** : Combinaison de plusieurs modèles avec un méta-modèle

Nous avons évalué les performances de ces modèles sur un jeu de validation, puis sélectionné le meilleur modèle pour l'évaluer sur le jeu de test. Nous avons également analysé l'importance des caractéristiques pour comprendre quels facteurs influencent le plus la satisfaction des passagers.

Les techniques d'Ensemble Learning ont montré leur efficacité pour améliorer les performances de classification par rapport aux modèles individuels. Le modèle qui a obtenu les meilleures performances est le [Insérer le nom du meilleur modèle ici] avec un F1 Score de [Insérer le F1 Score ici] sur le jeu de test.

Ces résultats démontrent l'intérêt des méthodes d'ensemble pour améliorer la robustesse et la précision des modèles de classification.